# Geolocation Model

Inputs:
- GPS coordinates: Real-time location data from GPS-enabled devices.
- Location data: Aggregated from mapping services and user inputs.
- Maps: Sourced from mapping APIs and geospatial data services.

Outputs:
- Precise location information: Real-time location tracking and updates.
- Geospatial analysis: Insights and analytics based on geographical data.

Usage:
- Navigation: Real-time tracking and routing.
- Emergency Assistance: Quickly locate users in need of help.
- Personalized Recommendations: Suggest nearby points of interest based on user location.

---

For more advanced geospatial analysis, we can integrate APIs (like Google Maps) to get detailed maps and additional location data, apply machine learning models for predictive analytics.

In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Point, Polygon # for geometry operations
import folium # for mapping
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

Generating Sample data

In [2]:
# Some major cities in India with their coordinates
data = {
    'City': ['Mumbai', 'Delhi', 'Bangalore', 'Hyderabad', 'Ahmedabad', 'Chennai', 'Kolkata', 'Surat', 'Pune', 'Jaipur',  'Nagpur', 'Bhopal', 'Lucknow', 'Kanpur', 'Indore', 'Patna', 'Ranchi', 'Raipur'],
    'Latitude': [19.0760, 28.7041, 12.9716, 17.3850, 23.0225, 13.0827, 22.5726, 21.1702, 18.5204, 26.9124, 21.1458, 23.2599, 26.8467, 26.4499, 22.7196, 25.5941, 23.3441, 21.2514],
    'Longitude': [72.8777, 77.1025, 77.5946, 78.4867, 72.5714, 80.2707, 88.3639, 72.8311, 73.8567, 75.7873, 79.0882, 77.4126, 80.9462, 80.3319, 75.8577, 85.1376, 85.3096, 81.6296]
}
df = pd.DataFrame(data)

# Additional points around these cities
np.random.seed(0)
num_additional_points = 300
additional_latitudes = np.random.uniform(df['Latitude'].min(), df['Latitude'].max(), num_additional_points)
additional_longitudes = np.random.uniform(df['Longitude'].min(), df['Longitude'].max(), num_additional_points)

additional_points = pd.DataFrame({
    'City': [''] * num_additional_points,
    'Latitude': additional_latitudes,
    'Longitude': additional_longitudes
})

# Combine the datasets
df = pd.concat([df, additional_points], ignore_index=True)

# removing points falling into seas approximately
def is_within_boundary(row):
    return not(row['Latitude']<16 and row['Longitude']>80.277) and not(row['Latitude']<17 and row['Longitude']<74) and not(row['Latitude']<22 and row['Longitude']<72.877) and not(row['Latitude']<19 and row['Longitude']>81.63) and not(row['Latitude']<22 and row['Longitude']>85.3)
    

df = df[df.apply(is_within_boundary, axis=1)].reset_index(drop=True)

print(df.shape)
df.head()


(247, 3)


,City,Latitude,Longitude
0,Mumbai,19.0760,72.8777
1,Delhi,28.7041,77.1025
2,Bangalore,12.9716,77.5946
3,Hyderabad,17.3850,78.4867
4,Ahmedabad,23.0225,72.5714


Geospatial Analysis

In [3]:
# Create points from the latitude and longitude
points = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]

centroid = Polygon([(p.x, p.y) for p in points]).centroid
print(f"Centroid: {centroid}")
buffers = [p.buffer(0.5) for p in points]

m = folium.Map(location=[centroid.y, centroid.x], zoom_start=5)

# Add points to the map
for point in points:
    folium.Marker([point.y, point.x]).add_to(m)

# Add buffers to the map
for buffer in buffers:
    folium.GeoJson(buffer.__geo_interface__).add_to(m)

m.save("india_map.html")

Centroid: POINT (80.14851914934005 21.21865191608361)


Clustering

In [4]:
coords = np.array([[point.y, point.x] for point in points])

kmeans = KMeans(n_clusters=8, random_state=0).fit(coords)
df['Cluster'] = kmeans.labels_

colors = ['cadetblue','gray','blue', 'purple','green','orange','black','darkred']
m_clustered = folium.Map(location=[centroid.y, centroid.x], zoom_start=5)

for i, point in enumerate(points):
    folium.Marker([point.y, point.x], icon=folium.Icon(color=colors[df.loc[i, 'Cluster']])).add_to(m_clustered)

# Save the clustered map to an HTML file
m_clustered.save("clustered_india_map.html")

c:\Users\bhara\anaconda3\envs\nullclass\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [5]:
m_clustered